# AST 7939 Homework Assignment #3 (10 pts)

Savannah Gramze

Use photometric data with different filters to infer the redshift.

If we have photometry and spectroscopy for certain galaxies, we can use machine learning to train to infer the redshifts of certain galaxies using photometry.

## All work is due Monday March 6 at 5 pm.

## Instruction: 
Do all homework in this Jupyter notebook and submit your final .ipynb file via Canvas. Show ALL your work and try to add comment lines as needed to describe what your code does. 

You are encouraged to discuss homework problems with your classmates. However, your python script and answers to the questions must be written by yourself.

## Regression with ensemble methods

### 1. Photometric redshift estimator using Random Forest (7 pts)

Following [Zhou et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019MNRAS.488.4565Z/abstract), the goal of this homework is to estimate photometric redshifts starting from observations of galaxy magnitudes in six different photometric bands (u, g, r, i, z, y), and compare the estimated photometric redshifts with spectroscopic redshifts. The data used in the paper are available at this [link](http://d-scholarship.pitt.edu/36064/). This example is motivated by an example in Machine Learning techniques for Physics and Astronomy that is being written by Dr. Viviana Acquaviva.

Redshift is a crucial observable in the study of galaxies and cosmology. Spectroscopic redshifts are accurate, but the observations required are much more expensive than photometric measurements. Modern imaging surveys can measure the photometry of a huge number of objects very efficiently, but only a very small fraction will have observed spectra. For such surveys, redshifts must be estimated from broad-band photometry, and the large number of photometric redshift measurements compensates for their inaccuracy. The availability of large imaging data sets has made photometric redshift estimates an increasingly important component of modern extragalactic astronomy and cosmology studies.



The paper used ugriz photometry from Canada–France–Hawaii Telescope Legacy Survey (CFHTLS) and Y-band photometry from the Subaru Suprime camera, as well as spectroscopic redshifts from the DEEP2, DEEP3, and 3D-HST surveys. In this homework problem, let's use DEEP2/3 data stored in DEEP2_uniq_Terapix_Subaru_v1.fits. You can read in the fits file by executing the cell below. See README file for more information on the content of the fits file.

In [1]:
import astropy
from astropy.io import fits
import pandas as pd
import numpy as np

with fits.open('./DEEP2_uniq_Terapix_Subaru_v1.fits') as data:
    df = pd.DataFrame(np.array(data[1].data).byteswap().newbyteorder()) 

(a) As in almost all real-life ML problems, start with data cleaning. Follow the steps below. (1 pt)

1. Collect deep field CFHTLS data only by choosing the objects with `cfhtls_source` = 0.
2. Collect deep source subaru data only by choosing the objects with `subaru_source` = 0.
3. Collect objects with redshift quality code `zquality` >= 3.
4. Collect objects with `u_apercor` < 99.
5. Collect objects with `y_apercor` < 99.

The 'features' we will use are aperture corrected ugrizy magnitudes, `u_apercor, g_apercor, r_apercor, i_apercor, z_apercor, y_apercor`. The 'label' we will use is the spectroscopic redshift `zhelio`. Put the features in a dataframe X and labels in a dataframe y. You should end up with 5874 objects after data cleaning. Split the data into training and test datasets.

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
cfdtls_data = df[df['cfhtls_source'] == 0]#.keys()
deep_source = cfdtls_data[cfdtls_data['subaru_source'] == 0]
quality_red = deep_source[deep_source['zquality'] >= 3.]
clean_u = quality_red[quality_red['u_apercor'] < 99]
clean_df = clean_u[clean_u['y_apercor'] < 99]

X = clean_df[['u_apercor', 'g_apercor', 'r_apercor', 'i_apercor', 'z_apercor', 'y_apercor']]
y = clean_df['zhelio']

X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y))

(b) Make a `RandomForestRegressor` model. Use the default hyperparameters for now (we will optimize them later). Compute the accuracy of the model in two ways. (1 pt)

1. Compute the training and test scores of the model using 5-fold cross validation. You will get 5 scores. Report the average of the 5 scores.
2. Compute the out-of-bag test score. 

In [13]:
# every time you bootstrap, there are some data left in original dataset
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
#KFold?
#RandomForestRegressor?

In [14]:
model = RandomForestRegressor(oob_score=True)

In [15]:
model.fit(X_train, y_train)
model.oob_score_ 

0.7286793325347354

In [12]:
kfold = KFold(n_splits = 5, shuffle = True)

In [15]:
for train, test in kfold.split(X_train):
    print('train: %s, test: %s' % (X_train[train], X_train[test]))

train: [[23.14385455 22.57061078 22.07681547 21.96614953 21.88239458 21.86940264]
 [22.90808034 21.69142985 20.34757164 19.52116763 19.16546715 19.02539304]
 [23.92743724 23.83899934 23.65775313 23.37988912 22.83812948 22.67727385]
 ...
 [23.00222627 22.83969551 22.24057104 21.93963208 21.99231725 21.84361009]
 [22.95836299 22.80062735 22.637572   22.10905323 21.9311262  22.28831105]
 [25.2195741  24.81068257 24.12032687 23.83355862 23.71999111 23.75485062]], test: [[23.42081552 23.17718898 23.09128725 22.88009722 22.7546036  22.30554184]
 [24.88916427 24.72872719 24.10570779 23.76448347 23.7949746  24.03178548]
 [24.49023882 24.23167237 23.55026683 23.07315501 22.94451298 22.86465101]
 ...
 [23.15444628 21.77713988 20.45626145 19.90422633 19.6271698  19.44926193]
 [23.57553899 23.58241547 23.42643695 23.20378427 22.8177214  23.61729998]
 [24.911186   24.47307877 23.6576746  22.98649944 22.30333954 22.03850741]]
train: [[23.14385455 22.57061078 22.07681547 21.96614953 21.88239458 21.86

(c) Using the `RandomForestRegressor` you built in #b, make predictions for the test dataset. This prediction will be the "photometric redshift". Then, make a scatter plot showing photometric redshift vs. spectroscopic redshift. Your figure should look similar to Figure 5 of Zhou et al. (2019). Also, compute the outlier fraction $\eta$, which is defined as the fraction of objects with $|z_{phot}-z_{spec}| > 0.15/(1+z_{spec})$. (1 pt)

In [ ]:
# make predictions for test dataset
# make a plot showing inferred photometric redshift vs actual spectroscopic redshift
# 6% of the data are outliers in the paper
# How much of the data are outliers?









(d) Let's try to optimize hyperparameters `max_features` and `n_estimators`. For `max_features`, try three different values: None, "sqrt", and 2. Compute the test scores of the best model and compare them with what you got in #b. (1 pt)

In [ ]:
# optimize max_features in random forest
# try None, sqrt and 2 for max_features (# of dimensions?)













(e) You might have found that the test scores are not drastically improved. What should we do? Let's try to add u-g, g-r, r-i, i-z, z-y colors and see if it improves the performance. You now have a 11-dimensional dataset. Using this new dataset, make a Random Forest, optimize hyperparameters, and compute the test score of the best model. (1 pt)

In [ ]:
# 70-80%, not super good
# to try to improve, add more features, ie colors
# make 5 colors and add them for 11 dimensions 










(f) With your best model, repeat #c. Make a plot showing photometric redshift vs. spectroscopic redshift, and compute the outlier fraction. Do you find better perfomance? (1 pt)

(g) Make a plot showing feature importance. Which is more important between colors and magnitudes? Does the feature importance plot support that adding colors helped improve the performance? (1 pt)

In [ ]:
# show feature importance bar graph 
# which are the most important features?











### 2. Photometric redshift estimator using Gradient Boosting (3 pts)


This time, let's use Gradient Boosting to compute photometric redshift. 

(a) Let's use `max_depth=2`. What `n_estimator` should we use? Make a plot showing the validation error vs. n_estimator and find the optimal `n_estimator`. What is your optimal `n_estimator` and what is the test score using the optimal model? (1 pt)

In [ ]:
# boosting, gradient
# use decision tree with max_depth=2
# can use as many n_estimator as you want, but what is optimal?
# find the best gradient boosting model










(b) Using the `GradientBoostingRegressor` you built in #a, make predictions for the test dataset. This prediction will be the "photometric redshift". Then, make a scatter plot showing photometric redshift vs. spectroscopic redshift. Your figure should look similar to Figure 5 of Zhou et al. (2019). Also, compute the outlier fraction $\eta$, which is defined as the fraction of objects with $|z_{phot}-z_{spec}| > 0.15/(1+z_{spec})$. (1 pt)

In [ ]:
# compare with random forest












(c) Make a movie showing the evolution of photometric redshift vs. spectroscopic redshift as you increase `n_estimator`. See gbr.mp4 on Canvas as an example. Discuss what you see in the movie in the context of boosting. Submit your movie file. (1 pt)

In [ ]:
# make a movie??????
# make a png for each n_estimator and then combine into 1000 frame mp4











### Congratulations! You reproduced results presented in yet another journal paper. If you enjoyed the homework and would like to do some additional related analysis, I recommend you have a look at the following papers and references therein. Feel free to read/reproduce them for paper presentation and/or term project.

https://ui.adsabs.harvard.edu/abs/2010ApJ...712..511C/abstract

https://ui.adsabs.harvard.edu/abs/2021MNRAS.505.4847H/abstract

https://ui.adsabs.harvard.edu/abs/2022MNRAS.509.2289L/abstract

https://ui.adsabs.harvard.edu/abs/2022MNRAS.512.1696H/abstract